This notebook hosts some ideas for how to set up a model specification in our OSE projects.

In [8]:
from collections import namedtuple
import yaml

import numpy as np

# TODO: add naming conventions

# This is where once and for all the model parameters are defined.
fields = ['rho', 'periods']

class ModelCls(namedtuple('model_base', ' '.join(fields))):
 
    def __eq__(self, other):
        assert isinstance(other, type(self))
        assert set(spec_1._fields) == set(spec_2._fields)
        for field in self._fields:
            if getattr(self, field) != getattr(other, field):
                return False
        return True
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __repr__(self):
        str_ = ''
        for field in self._fields:
            str_ +='{:}: {:}\n'.format(field, getattr(self, field))
        return str_
    
    def as_dict(self):
        return self._asdict()
    
    def to_yaml(self, fname='test.yml'):
        with open(fname, 'w') as out_file:
            yaml.dump(self._asdict(), out_file)

def generate_random_init(constr):
    init_dict = dict()
    init_dict['rho'] = np.random.uniform(0.01, 0.99)
    init_dict['periods'] = np.random.randint(1, 10)
    
    if 'periods' in constr.keys():
        init_dict['periods'] = constr['periods']

    if 'rho' in constr.keys():
        init_dict['rho'] = constr['rho']
        
    return init_dict
    
    
def get_model_obj(input_, constr=None):
    """This is a factory method to create a model spefication from a variety of inputs."""    
    if isinstance(input_, dict):
        return ModelCls(**init_dict)
    elif isinstance(input_, str):
        return ModelCls(**yaml.load(open(input_, 'r'), Loader=yaml.FullLoader))
    elif input_ is None:
        return ModelCls(**generate_random_init(constr))
    else:
        raise NotImplementedError

init_dict = dict()
init_dict['rho'] = 0.5
init_dict['periods'] = 2

for input_ in [init_dict, 'model_spec.yml']:
    get_model_obj(input_)
    
spec_1 = get_model_obj(init_dict)
spec_2 = get_model_obj('model_spec.yml')

spec_1.to_yaml()

# This simulates the adapter to the optimizer.
spec_3 = spec_1._replace(rho=0.9)
spec_3 = spec_1

spec_1 == spec_2

constr = dict()
constr['rho'] = 5
spec_4 = get_model_obj(None, constr)
print(spec_4)

rho: 5
periods: 6



## Use cases 

We want to explore some use cases with a basic model and translate them into tests.